In [5]:
from utils import client

PrimaryKeyException: <PrimaryKeyException: (code=1, message=Primary key type must be DataType.INT64 or DataType.VARCHAR.)>

In [6]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, utility
from pymilvus import connections

conn = connections.connect(
    host="ii578293.iptime.org", 
    port=19530
)

In [14]:
import json

with open("./utils/field_args.json", 'r') as f:
    cfg = json.load(f)

In [15]:
COLLECTION_NAME = cfg['COLLECTION_NAME']
DIMENSION = cfg['DIMENSION']
INDEX_TYPE = cfg['INDEX_TYPE']

if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

In [16]:
field_name = {
        "file_id" : {
        "dtype" : DataType.INT64, 
        "is_primary" : True, 
        "auto_id" : True
    },
    "filename" : {
        "dtype" : DataType.VARCHAR, 
        "max_length" : 20
    },
    "date" : {
        "dtype" : DataType.VARCHAR,
        "max_length" : 10
    },
    "Title" : {
        "dtype" : DataType.VARCHAR, 
        "max_length" : 1000
    },
    "Reviews" : {
        "dtype" : DataType.VARCHAR, 
        "max_length" : 1000
    },
    "Embeddings" : {
        "dtype" : DataType.FLOAT_VECTOR, 
        "dim" : 768
    },
    cfg["EMBEDDING_FIELD_NAME"] : {
            "dtype" : DataType.FLOAT_VECTOR,
            "dim" : cfg["DIMENSION"]
        }
}
for n, args in field_name.items():
    print(n, args)
    FieldSchema(name = n, **args)
    break

file_id {'dtype': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}


In [27]:
from pymilvus import IndexType
# 직접 밀어넣을 필드 & 스키마 정의하기
fields = [
    FieldSchema(name = n, **args) for n, args in field_name.items()
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

if collection.has_index():
    try:
        collection.drop_index()
    except:
        collection.release()
        collection.drop_index()

# 중요한 인덱스 부분
index_params = {
    'metric_type':'L2',
    'index_type': INDEX_TYPE,
    'params':{"nlist":128}
}

# colletion에서 색인 부분을 생성하는 부분을 명시하고 부르겠다는 뜻
collection.create_index(
    field_name=cfg["EMBEDDING_FIELD_NAME"], index_params=index_params
)
collection.load()
collection

2025-02-03 23:49:32,496 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=1, message=cannot create index on non-existed field: qna)>, <Time:{'RPC start': '2025-02-03 23:49:32.491199', 'RPC error': '2025-02-03 23:49:32.496238'}> (decorators.py:140)


MilvusException: <MilvusException: (code=1, message=cannot create index on non-existed field: qna)>

In [26]:
collection.index()
collection.has_index()

True

In [ ]:
db.list_database()

from pymilvus import MilvusClient
from pymilvus import MilvusClient, DataType


_URI = "http://localhost:19530"
_TOKEN = "root:EncoreProject4"
_DB_NAME = "project4"

class db_connection:
    def __init__(self, uri, token, database):
        self.client = MilvusClient(uri=uri, db_name=database, token=token)

    def get_client(self):
        return self.client

    def create_schema(self, auto_id=False, enable_dynamic_field=True):
        schema = self.client.create_schema(
            auto_id=auto_id,
            enable_dynamic_field=enable_dynamic_field,
        )

        return schema

    def insert_data(self, collection_name, data):
        res = self.client.insert(
            collection_name=collection_name,
            data=data
        )
        
        return res
    
connect = db_connection(_URI, _TOKEN, _DB_NAME)

In [ ]:
database = db.create_database("project4")

schema = connect.create_schema()

# 3.2. Add fields to schema
schema.add_field(field_name="review_id6", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="review_vector6", datatype=DataType.FLOAT_VECTOR, dim=384)
schema.add_field(field_name="review_varchar6", datatype=DataType.VARCHAR, max_length=8192)

# 3.3. Prepare index parameters
index_params = client.prepare_index_params()

# 3.4. Add indexes
index_params.add_index(
    field_name="review_id6",
    index_type="STL_SORT"
)

index_params.add_index(
    field_name="review_vector6", 
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

# 3.5. Create a collection with the index loaded simultaneously
client.create_collection(
    collection_name="review_data",
    schema=schema,
    index_params=index_params
)

res = client.get_load_state(
    collection_name="review_data"
)

print(res)

import pandas as pd

milk_df = pd.read_csv('./data/milk.csv', delimiter=",")

def change_float_vector(tensor):
    return tensor.float().tolist()

insert_data = []
LIMIT_DATA = 500

cnt = 0
for index , (star, review) in tqdm(milk_df.iterrows()):
    #print(key)
    #print(value)
    insert_data.append({'review_id2' : index, 
                        'review_vector2' : change_float_vector(get_sentence_embedding(review)),
                        'review_varchar2' : review})

    #print(get_sentence_embedding(review))
    #print(change_float_vector(get_sentence_embedding(review)))

    cnt += 1
    if cnt == LIMIT_DATA:
        break

re_data2 = []

for _data in insert_data:
    _data123 = {}
    _data123['review_vector6'] = _data['review_vector2']
    _data123['review_varchar6'] = _data['review_varchar2']
    _data123['review_id6'] = _data['review_id2']
    re_data2.append(_data123)

print(re_data2[0])

res = connect.get_client().insert( collection_name="review_data",data=re_data2)
